In [507]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy.typing as npt
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

In [508]:
RANDOM_STATE=42
TEST_SIZE=0.2

In [509]:
df: pd.DataFrame = pd.read_csv("data/folha6.csv")
df.dropna(axis="columns", thresh=5000, inplace=True)
df = df[df["UnitLengthStay"].apply(lambda x: x <= 24)] # deixando apenas os q ficaram ate um dia no unit
df.fillna(value=0, inplace=True)
discharge: pd.Series = (df.loc[:, df.columns == "UnitDischargeName"] == "Óbito").astype(
    np.int8
)
discharge.columns = ["DeathInUnit"]
fisiologic_data: pd.DataFrame = df.iloc[:, 21:].copy()
""" fisiologic_data = fisiologic_data.div(
    fisiologic_data.sum(axis=1), axis=0
) """
labels: pd.DataFrame = df["UnitLengthStay"].copy()
labels = pd.cut(labels, bins=[0, 2, 24], labels=np.arange(2))

In [510]:
X_train, X_test, y_train, y_test = train_test_split(
    fisiologic_data, labels, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

In [511]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(18,)),
        tf.keras.layers.Dense(units=12, activation="sigmoid"),
        tf.keras.layers.Dense(units=1, activation="softmax"),
    ]
)

loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
metrics = [tf.keras.metrics.BinaryAccuracy()]

model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [512]:
model_history = model.fit(
    X_train, y_train, batch_size=100, epochs=10, shuffle=True, verbose=1
)

y_pred = model.predict(x=X_test)
y_pred = y_pred.argmax(axis=-1)
f1_score(y_test, y_pred, average=None)

Epoch 1/10


57/57 [==============================] - 0s 2ms/step - loss: 0.6887 - binary_accuracy: 0.4477
Epoch 2/10
57/57 [==============================] - 0s 2ms/step - loss: 0.6736 - binary_accuracy: 0.4477
Epoch 3/10
57/57 [==============================] - 0s 2ms/step - loss: 0.6759 - binary_accuracy: 0.4477
Epoch 4/10
57/57 [==============================] - 0s 3ms/step - loss: 0.6734 - binary_accuracy: 0.4477
Epoch 5/10
57/57 [==============================] - 0s 1ms/step - loss: 0.6832 - binary_accuracy: 0.4477
Epoch 6/10
57/57 [==============================] - 0s 1ms/step - loss: 0.6820 - binary_accuracy: 0.4477
Epoch 7/10
57/57 [==============================] - 0s 1ms/step - loss: 0.6762 - binary_accuracy: 0.4477
Epoch 8/10
57/57 [==============================] - 0s 2ms/step - loss: 0.6776 - binary_accuracy: 0.4477
Epoch 9/10
57/57 [==============================] - 0s 1ms/step - loss: 0.6798 - binary_accuracy: 0.4477
Epoch 10/10
44/44 [==============================] - 0s 1ms/step


array([0.70637119, 0.        ])

In [513]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X=X_train, y=y_train)
y_pred = rfc.predict(X_test)
f1_score(y_test, y_pred, average=None)


array([0.7327478 , 0.64735099])

In [514]:
df = pd.read_csv("data/folha3.csv")
ages = df["Age"].copy()
df = df[df["UnitLengthStay"].apply(lambda x: x <= 24)] # deixando apenas os q ficaram ate um dia no unit
df = df.iloc[:, 22:]
df.dropna(axis=1, thresh=5000, inplace=True)

comorbidades = (df == "Verdadeiro").astype(int)

df = pd.read_csv("data/folha4.csv")
df = df[df["UnitLengthStay"].apply(lambda x: x <= 24)] # deixando apenas os q ficaram ate um dia no unit
df = df.iloc[:, 21:]
df.dropna(axis=1, thresh=5000, inplace=True)
razoes = (df == "Verdadeiro").astype(int)

result = pd.concat([ages, razoes, comorbidades], axis=1, join="inner")

X_train, X_test, y_train, y_test = train_test_split(
    result, labels, test_size=0.3, random_state=RANDOM_STATE
)

rfc = RandomForestClassifier(n_estimators=300, criterion="log_loss")
rfc.fit(X=X_train, y=y_train)
y_pred = rfc.predict(X_test)
f1_score(y_test, y_pred)


0.5373455131649651

In [516]:
import xgboost as xgb
model = xgb.XGBClassifier(n_estimators=3000)
model.fit(result, labels)
y_pred = model.predict(X_test)
display(confusion_matrix(y_test, y_pred))
f1_score(y_test, y_pred)

array([[1115,   31],
       [  82,  873]])

0.9392146315223238